In [2]:
## Code base used as reference: https://python.langchain.com/docs/integrations/providers/cohere/
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
model="command-xlarge"

## Cohere Embedding Model Options:  
Cohere has two main “families” of generative models—Command and Base—each with different sizes (light, medium, xlarge).
1. Command Family
    * command-light
    * command-medium
    * command-xlarge
    * (Sometimes referred to simply as command when using the default version)
2. Base Family
    * base-light
    * base-medium
    * base-xlarge
    * (Sometimes referred to simply as base when using the default version)

In [19]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere(model=model)
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages).content)

Who's there?


## LLM

In [20]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere(model=model)
messages = [HumanMessage(model)]
print(chat.invoke(messages).content)

I cannot directly access the internet in order to search for the model you are referencing. However, if you describe the functionality and use of this model, I may be able to elaborate on its relevance and applications.


In [21]:
from langchain_cohere.llms import Cohere

llm = Cohere()
print(llm.invoke("Come up with a pet name"))

 Of course! Here are some suggestions for pet names: 

- Sweetheart 
- Honeybunch 
- Darling 
- Cutie 
- Sweetie 
- Pumpkin 
- Dove 
- Snugglebug 

These are some gentle, affectionate names you could call someone you care about. You may choose one that you feel best represents your relationship with the recipient. 

Let me know if you would like me to provide additional options or more personalized suggestions! 


## Tool calling

In [22]:
from langchain_cohere import ChatCohere
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
)
from langchain_core.tools import tool

@tool
def magic_function(number: int) -> int:
    """Applies a magic operation to an integer

    Args:
        number: Number to have magic operation performed on
    """
    return number + 10

def invoke_tools(tool_calls, messages):
    for tool_call in tool_calls:
        selected_tool = {"magic_function":magic_function}[
            tool_call["name"].lower()
        ]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    return messages

tools = [magic_function]

llm = ChatCohere()
llm_with_tools = llm.bind_tools(tools=tools)
messages = [
    HumanMessage(
        content="What is the value of magic_function(2)?"
    )
]

res = llm_with_tools.invoke(messages)
while res.tool_calls:
    messages.append(res)
    messages = invoke_tools(res.tool_calls, messages)
    res = llm_with_tools.invoke(messages)

print(res.content)

The value of magic_function(2) is **12**.


## ReACT

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

llm = ChatCohere()

internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [internet_search],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

agent_executor.invoke({
    "input": "In what year was the company that was founded as Sound of Music added to the S&P 500?",
})



> Entering new AgentExecutor chain...

I will search for 'In what year was the company that was founded as Sound of Music added to the S&P 500?'
{'tool_name': 'internet_search', 'parameters': {'query': 'In what year was the company that was founded as Sound of Music added to the S&P 500?'}}
[{'url': 'https://www.companieshistory.com/best-buy/', 'content': "The company changed its name from Sound of Music, Inc. to Best Buy Co., Inc. in 1983. Best Buy was founded by Richard M. Schulze in 1966 and is headquartered in Richfield, MN. ... In 1999, Best Buy was added to Standard & Poor's S&P 500. 2000s. In 2000, Best Buy formed Redline Entertainment, an independent music label and action-sports video"}, {'url': 'https://historydraft.com/story/best-buy/timeline/841', 'content': 'Best Buy Co., Inc. is an American multinational consumer electronics retailer headquartered in Richfield, Minnesota. Originally founded by Richard M. Schulze and James Wheeler in 1966 as an audio specialty store call

{'input': 'In what year was the company that was founded as Sound of Music added to the S&P 500?',
 'output': 'Best Buy Co., Inc. was added to the S&P 500 in 1999. The company was founded as Sound of Music in 1966 and changed its name to Best Buy in 1983.',
 'citations': [CohereCitation(start=0, end=18, text='Best Buy Co., Inc.', documents=[{'url': 'https://www.companieshistory.com/best-buy/', 'content': "The company changed its name from Sound of Music, Inc. to Best Buy Co., Inc. in 1983. Best Buy was founded by Richard M. Schulze in 1966 and is headquartered in Richfield, MN. ... In 1999, Best Buy was added to Standard & Poor's S&P 500. 2000s. In 2000, Best Buy formed Redline Entertainment, an independent music label and action-sports video", 'id': 'doc_0'}, {'url': 'https://historydraft.com/story/best-buy/timeline/841', 'content': 'Best Buy Co., Inc. is an American multinational consumer electronics retailer headquartered in Richfield, Minnesota. Originally founded by Richard M. Sch

## Cohere Embedding Model Options:  
- embed-english-v2.0 : General-purpose embeddings for English text.

- embed-english-light-v2.0
A lighter-weight model for English, optimized for speed and smaller resource usage.

- embed-multilingual-v2.0
Designed for multilingual text (supporting multiple languages), enabling cross-lingual semantic tasks.

- embed-multilingual-light-v2.0
A lighter-weight multilingual model, similar to embed-english-light-v2.0 but supporting multiple languages.

In [25]:
from langchain_cohere import ChatCohere
from langchain.retrievers import CohereRagRetriever
from langchain_core.documents import Document

rag = CohereRagRetriever(llm=ChatCohere(model=model), model="embed-english-v2.0")
print(rag.invoke("What is cohere ai?")[0].page_content)

/Users/denizaskin/CodeBase/agenticchunking/.venv/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py:851: LangChainDeprecationWarning: The 'connectors' parameter is deprecated as of version 0.3.3.
Please use the 'tools' parameter instead.
  result = self._generate(


Cohere Inc. is a Canadian multinational technology company focused on artificial intelligence for the enterprise, specializing in large language models. Cohere was founded in 2019 by Aidan Gomez, Ivan Zhang, and Nick Frosst, and is headquartered in Toronto and San Francisco, with offices in Palo Alto, London, and New York City.

In 2017, a team of researchers at Google Brain introduced the transformer machine learning architecture in "Attention Is All You Need," which demonstrated state-of-the-art performance on a variety of natural language processing tasks. In 2019, Aidan Gomez, one of its co-authors, along with Nick Frosst, another researcher at Google Brain, founded Cohere with Ivan Zhang, with whom Gomez had done research at FOR.ai. All of the co-founders attended University of Toronto.


In [6]:
# pip install cohere
import cohere
co = cohere.ClientV2(api_key="COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

# Define the documents
faqs_short = [
    {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
    {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."},
    {"text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."},
    {"text": "Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year."}
]

# Add the user query
query = "Are there fitness-related perks?"
# Rerank the documents
results = co.rerank(query=query,
                    documents=faqs_short,
                    top_n=2,
                    model='rerank-english-v3.0')
print(results)

# Display the reranking results
def return_results(results, documents):    
    for idx, result in enumerate(results.results):
        print(f"Rank: {idx+1}") 
        print(f"Score: {result.relevance_score}")
        print(f"Document: {documents[result.index]}\n")
    
return_results(results, faqs_short)

UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}